In [ ]:
#Only required when running on gcloud datalab
!pip install keras
!pip install --update tensorflow

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import keras
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, CuDNNLSTM, Input, concatenate, Concatenate
from keras.backend import clear_session
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
# Predictor data from Professor Amit Goyal's Website (Welch and Goyal (2008))
url = 'http://www.hec.unil.ch/agoyal/docs/PredictorData2017.xlsx'
mn_df = pd.read_excel(url,index_col='yyyymm')

In [ ]:
# Data Loading Snippets from Google Cloud Datastorage
# <Only used when running on Google Cloud Datalab>
from google.datalab import storage
from io import BytesIO
mybucket = storage.Bucket('master_thesis_storage')
data_ori = mybucket.object('mn_df_ori.csv')
data_sqr = mybucket.object('mn_df_sqr.csv')
data_asy = mybucket.object('mn_df_asy.csv')
uri_ori = data_ori.uri
uri_sqr = data_sqr.uri
uri_asy = data_asy.uri
%gcs read --object $uri_ori --variable data_ori
mn_df_ori = pd.read_csv(BytesIO(data_ori))
%gcs read --object $uri_sqr --variable data_sqr
mn_df_sqr = pd.read_csv(BytesIO(data_sqr))
%gcs read --object $uri_asy --variable data_asy
mn_df_asy = pd.read_csv(BytesIO(data_asy))
mn_df_ori.set_index('yyyymm', inplace=True)
mn_df_sqr.set_index('yyyymm', inplace=True)
mn_df_asy.set_index('yyyymm', inplace=True)

In [4]:
# This cell contains all the dataset preparation, customized function
def variable_deriv(df):    
    #Yielding Equity Premium    
    df['premium'] = np.log((df.Index+df.D12)/(df.Index.shift(periods=1)*(1+df.Rfree)))
    #Yielding Indicator where its previous Return was positive
    df['indicator'] = (df.premium.shift(periods=1)>0).apply(lambda x: 1 if x==True else 0)       
    #Yielding Dividend Price Ratio
    df['dp'] = np.log(df.D12) - np.log(df.Index)
    #Yielding Divident Yield
    df['dy'] = np.log(df.D12) - np.log(df.Index.shift(periods=1))
    #Yielding Earnings Price Ratio
    df['ep'] = np.log(df.E12) - np.log(df.Index)
    #Yielding Divident Payout Ratio
    df['de'] = np.log(df.D12) - np.log(df.E12)
    #Yielding The Term Spread
    df['tms'] = df.lty - df.tbl
    #Yielding The Default Yield Spread
    df['dfy'] = df.BAA - df.AAA
    #Yielding The Default Return Spread
    df['dfr'] = df.corpr - df.ltr
################################################################################################################
variable_deriv(mn_df)
################################################################################################################
variables = ['dp','dy','ep','de','svar','bm','ntis','tbl','ltr','tms','dfy','dfr','infl']
all_variables= ['premium','dp','dy','ep','de','svar','bm','ntis','tbl','ltr','tms','dfy','dfr','infl']
variables_sq=variables.copy()
variables_sq.extend([item+str('_sq') for item in variables])
variables_asy=variables.copy()
variables_asy.extend([item+str('_ind') for item in variables])
################################################################################################################
##Getting final variables from 1927.01~2017.12
def variable_transform(df, type_=None):
    ori=df.loc[192701:,all_variables]
    ind=df.loc[192701:,'indicator']
    #X 들
    if type_=="original" :
        ori.loc[:,variables] = StandardScaler().fit_transform(ori.loc[:,variables])
        return ori
    #X^2 들
    elif type_=="squared":
        output = pd.concat([ori,ori.apply(lambda x: x**2).add_suffix('_sq')],axis=1)
        output.loc[:,variables_sq] = StandardScaler().fit_transform(output.loc[:,variables_sq])
        return output
    #Asymmetric term
    elif type_=="asymmetric":    
        output = pd.concat([ori,ori.apply(lambda x: x*ind).add_suffix('_ind')],axis=1)
        output.loc[:,variables_asy] = StandardScaler().fit_transform(output.loc[:,variables_asy])
        return output
################################################################################################################
mn_df_ori=variable_transform(mn_df, type_='original')
mn_df_sqr=variable_transform(mn_df, type_='squared')
mn_df_asy=variable_transform(mn_df, type_='asymmetric')
################################################################################################################
# Defining Generator for Time-Series Cross-Validation
class timeseries_cv():
    
    def __init__(self, n_split=3) :
        self.n_split = n_split
        
    def split(self, X, y, group=None) :
        n_split = self.n_split
        n_fold = n_split + 1
        max_len = X.shape[0]
        indices = np.arange(max_len)
        for index in range(1,n_fold) :
            yield (indices[:-index],indices[[-index]])
    
    def get_n_splits(self, X, y, group=None):
        return self.n_split    
################################################################################################################
# Defining RMSE function for Keras (Ultimately, MSE was used instead)
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))
################################################################################################################
def mse(y_true, y_pred):
    return np.average(np.square(y_true[[0]],y_pred[[0]]))
################################################################################################################
# Out-of-sample R-2 Coefficient proposed by Campbell and Thompson (2007)
def r2_os(y_true,y_pred):
    r_value = pd.DataFrame(index=y_pred.index, columns=['numerator','denominator'])
    r_value['numerator'] = (y_true - y_pred)**2
    r_value['denominator'] = (y_true - y_true.expanding(min_periods=0).mean())**2
    r_square = 1 - (r_value['numerator'].sum())/(r_value['denominator'].sum())
    return r_square

### LSTM 모델

In [5]:
# MLP Layer Constructing with concatenation of pen-ultimate hidden layer with initial inputs
def mlp_merge_builder(hidden_units, input_shape_arg):
    main_input = Input(shape=(input_shape_arg,), dtype='float32', name='main_input')
    raw_input = Input(shape=(input_shape_arg,), name='concat_input')

    for i in range(len(hidden_units)):
        # First layer 
        if len(hidden_units)==1:
            x = Dense(hidden_units[0], activation='relu',name=str(hidden_units[0]), kernel_regularizer=regularizers.l2(0.001))(main_input)
            x = Dropout(0.5)(x)
            x = concatenate([x, raw_input])
            break
        elif i == 0 :
            x = Dense(hidden_units[0], activation='relu',name=str(hidden_units[0]), kernel_regularizer=regularizers.l2(0.001))(main_input)
            x = Dropout(0.5)(x)            
        # Final layer (concatenation)
        elif i == (len(hidden_units)-1):
            x = concatenate([x, raw_input])
            x = Dense(hidden_units[i], activation='relu', name=str(hidden_units[i]), kernel_regularizer=regularizers.l2(0.001))(x)
            x = Dropout(0.5)(x)
        # Hidden layer
        else:
            x = Dense(hidden_units[i], activation='relu', name=str(hidden_units[i]), kernel_regularizer=regularizers.l2(0.001))(x)
            x = Dropout(0.5)(x)      
    
    predictions = Dense(1, activation='linear', name='output')(x)
    model = Model(inputs=[main_input, raw_input], outputs=[predictions])
    opt = keras.optimizers.SGD(lr=1e-3, decay = 1e-5, nesterov=True, momentum=0.9, clipnorm=5)
    model.compile(optimizer=opt, loss='mse',metrics=['mae'])
    
    return model

# LSTM Layer Constructing without concatenation of pen-ultimate hidden layer with initial inputs
def rnn_merge_builder(hidden_units, input_shape_arg):
    main_input = Input(shape=(1,input_shape_arg), dtype='float32', name='main_input')
    # Case when Only one LSTM layer is used
    if len(hidden_units)==1:
        x = CuDNNLSTM(hidden_units[0], name=str(hidden_units[0]), kernel_regularizer=regularizers.l2(0.001))(main_input)
        x = Dropout(0.2)(x)
    # Layer2개 이상일 경우
    elif len(hidden_units) != 1:
        for i in range(len(hidden_units)):
            # 첫 번째 layer 
            if i == 0 :
                x = CuDNNLSTM(hidden_units[0], return_sequences=True, name=str(hidden_units[0]))(main_input)
                x = Dropout(0.2)(x)            
            # 마지막 layer (concatenation)
            elif i == (len(hidden_units)-1):
                x = CuDNNLSTM(hidden_units[i], name=str(hidden_units[i]))(x)
                x = Dropout(0.2)(x)
            # 중간 layer
            else:
                x = CuDNNLSTM(hidden_units[i], return_sequences=True, name=str(hidden_units[i]))(x)
                x = Dropout(0.2)(x)      
    
    predictions = Dense(1, activation='linear', name='output')(x)
    model = Model(inputs=[main_input], outputs=[predictions])
    opt = keras.optimizers.RMSprop()
    model.compile(optimizer=opt, loss='mse',metrics=['mae'])
    
    return model    

# LSTM Layer Constructing with concatenation of pen-ultimate hidden layer with initial inputs
def rnn_merge_builder(hidden_units, input_shape_arg):
    main_input = Input(shape=(1,input_shape_arg), dtype='float32', name='main_input')
    # With only 1 LSTM layer, the LSTM layers returns output with reduced dimension. Therefore, when concatenating with initial
    # input, the dimension discrepancy raises error. To overcome this error, raw_input which is basically copy of initial input
    # is also declared with 2-dimensional data. Else, Raw-input is declared with 3-D shape.
    if len(hidden_units) ==1:
        raw_input = Input(shape=(input_shape_arg,), name='concat_input')
    else:
        raw_input = Input(shape=(1,input_shape_arg), name='concat_input')
    # Case when Only one LSTM layer is used 
    if len(hidden_units)==1:
        x = CuDNNLSTM(hidden_units[0], name=str(hidden_units[0]), kernel_regularizer=regularizers.l2(0.001))(main_input)
        x = Dropout(0.5)(x)
        x = concatenate([x, raw_input])
    # Case when more than one LSTM layers are used
    elif len(hidden_units) != 1:
        for i in range(len(hidden_units)):
            # first layer 
            if i == 0 :
                x = CuDNNLSTM(hidden_units[0], return_sequences=True, name=str(hidden_units[0]), kernel_regularizer=regularizers.l2(0.001))(main_input)
                x = Dropout(0.5)(x)            
            # last layer (concatenation)
            elif i == (len(hidden_units)-1):
                x = concatenate([x, raw_input])
                x = CuDNNLSTM(hidden_units[i], name=str(hidden_units[i]), kernel_regularizer=regularizers.l2(0.001))(x)
                x = Dropout(0.5)(x)
            # hidden layers
            else:
                x = CuDNNLSTM(hidden_units[i], return_sequences=True, name=str(hidden_units[i]), kernel_regularizer=regularizers.l2(0.001))(x)
                x = Dropout(0.5)(x)      
    
    predictions = Dense(1, activation='linear', name='output')(x)
    model = Model(inputs=[main_input, raw_input], outputs=[predictions])
    opt = keras.optimizers.RMSprop()
    model.compile(optimizer=opt, loss='mse',metrics=['mae'])
    
    return model

In [10]:
# Function for Calculating Predicted Premium
# Suppose to work on expanding and rolling window 
def nn_prediction(hidden_unit,name,df, type_, variables, window_type, window_range):
    ### Variables assignment for calling ###
    variables = ['dp','dy','ep','de','svar','bm','ntis','tbl','ltr','tms','dfy','dfr','infl']
    window_range = 12 * int(window_range)
    if type_=='original':
        input_shape_arg = 13
    elif type_=='squared':
        variables.extend([item+str('_sq') for item in variables])
        input_shape_arg = 26
    elif type_=='asymmetrical':
        variables.extend([item+str('_ind') for item in variables])
        input_shape_arg = 26
        
    pred_premium = pd.DataFrame(index=df.iloc[window_range+1:,:].index,columns=[name+'_{0}_pred'.format(str(hidden_unit)),name+'_{0}_r2'.format(str(hidden_unit))])
    
    for item in range(1091-window_range): 
        #item=0 denotes 192701         # item=1091 denotes 201712
        #############################################################################################################        
        print(item,"th repeating among {0}".format((1091-window_range)))
        if window_type == 'expanding':
            base = 0
        else :
            base = item
        ### Selecting Variables
        x_train = (df.iloc[(base):(item+window_range),df.columns.get_indexer(variables)]).values
        x_test = (df.iloc[(item+window_range):(item+(window_range+1)),df.columns.get_indexer(variables)]).values
        y_train = df['premium'].iloc[base+1:(item+(window_range+1))].values
        y_test = df['premium'].iloc[item+(window_range+1):(item+(window_range+2))].values
        #############################################################################################################                                  
        ### Setting Aside the last data as validation set for Early Stopping
        # When believed to be overfitted, turn on Early-Stopping 
        x_val, x_train = x_train[-1].reshape(1,-1), x_train[:-1]
        y_val, y_train = y_train[[-1]], y_train[:-1]
        #############################################################################################################        
        x_train_concat = x_train.copy()
        x_test_concat = x_test.copy()
        x_val_concat = x_val.copy()
        #############################################################################################################                          
        ### Training Selection (LSTM vs MLP)
        if name == 'LSTM':
            # LSTM Inputs should have Tensor 3D Shape
            x_train = x_train.reshape(x_train.shape[0],1,x_train.shape[1])
            y_train=y_train.reshape(y_train.shape[0],1)
            x_test = x_test.reshape(x_test.shape[0],1,x_test.shape[1])
            y_test = y_test.reshape(y_test.shape[0],1)
            x_val = x_val.reshape(x_val.shape[0],1,x_val.shape[1])
            y_val = y_val.reshape(y_val.shape[0],1)            
            # Concatenation에서 Layer1개만 있을 경우 CuDNNLSTM의 output이 축소되어 return 되기에 이와 concat해줄
            # raw input도 맞춰주기 위해서 x_train_concat가 조건만족시 reshape 적용하여 차원 확장
            if len(hidden_unit) == 1:
                pass
            elif len(hidden_unit) >= 1:
                x_train_concat = x_train_concat.reshape(x_train_concat.shape[0],1,x_train_concat.shape[1])
                x_test_concat = x_test_concat.reshape(x_test_concat.shape[0],1,x_test_concat.shape[1])
                x_val_concat = x_val_concat.reshape(x_val_concat.shape[0],1,x_val_concat.shape[1])
                
        elif name == 'MLP':
            # MLP does not require input to have Tensor 3D shape(No need to reshape)
            pass
        #############################################################################################################                                  
        ### Model Fitting
        early_stopping = EarlyStopping(monitor='val_loss',mode='min', patience=3, restore_best_weights=True)
        if name == 'LSTM':
            plan = rnn_merge_builder(hidden_unit,input_shape_arg)
            batch_size = x_train.shape[0]
            # Below history, model_pred, in_sample_pred are used when LSTM isn't construted with concatenation
            history = plan.fit({'main_input':x_train},{'output':y_train}, epochs=10, validation_data=([x_val],[y_val]),
                             batch_size=256, callbacks=[early_stopping], shuffle=False,verbose=0)
            model_pred = plan.predict([x_test])
            in_sample_pred = plan.predict([x_train])
            
        elif name == 'MLP':
            plan = mlp_merge_builder(hidden_unit,input_shape_arg)
            batch_size = x_train.shape[0]
            # Below history, model_pred, in_sample_pred are used when LSTM isn't construted with concatenation
            history=plan.fit({'main_input':x_train,'concat_input':x_train_concat},{'output':y_train}, epochs=10, validation_data=([x_val,x_val_concat],[y_val]),
                             batch_size=256, callbacks=[early_stopping], shuffle=False,verbose=0)
            model_pred = plan.predict([x_test,x_test_concat])
            in_sample_pred = plan.predict([x_train,x_train_concat])
        #############################################################################################################                                  
        #Below history is used when both LSTM and MLP are constructed using concatenation
        #history=plan.fit({'main_input':x_train,'concat_input':x_train_concat},{'output':y_train}, epochs=10, 
        #                 validation_data=([x_val,x_val_concat],[y_val]), batch_size=batch_size, shuffle=False,verbose=0)
        #model_pred = plan.predict([x_test,x_test_concat])
        #in_sample_pred = np.nan_to_num(plan.predict([x_train,x_train_concat]))
        #r2 = r2_score(y_true=y_train,y_pred=in_sample_pred)
        #############################################################################################################                                  
        ### plotting learning curve
#         plt.figure(dpi=150)
#         plt.plot(history.history['loss'], label='item_train_loss');
#         plt.plot(history.history['val_loss'], label='item_val_loss');
#         plt.ylabel('loss');
#         plt.xlabel('epoch');
#         plt.legend(loc='best');
#         plt.show()
        #############################################################################################################        
        ### Saving the prediction result         
        pred_premium.iloc[item,0] = model_pred[0,0]
        pred_premium.iloc[item,1] = r2
        clear_session()
        
    return pred_premium

In [ ]:
# lstm expanding 35yr (32,16,8) 
lstm_original_expanding_35yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_ori, 'original', variables, 'expanding', 35)
lstm_original_expanding_35yr_32_16_8.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_original_expanding_35yr_(32_16_8).csv')
lstm_squared_expanding_35yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_sqr, 'squared', variables_sq, 'expanding', 35)
lstm_squared_expanding_35yr_32_16_8.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_squared_expanding_35yr_(32_16_8).csv')
lstm_asymmetrical_expanding_35yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'expanding', 35)
lstm_asymmetrical_expanding_35yr_32_16_8.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_asymmetrical_expanding_35yr_(32_16_8).csv')

In [ ]:
# lstm expanding 35yr (32,16) 
lstm_original_expanding_35yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_ori, 'original', variables, 'expanding', 35)
lstm_original_expanding_35yr_32_16.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_original_expanding_35yr_(32_16).csv')
lstm_squared_expanding_35yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_sqr, 'squared', variables_sq, 'expanding', 35)
lstm_squared_expanding_35yr_32_16.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_squared_expanding_35yr_(32_16).csv')
lstm_asymmetrical_expanding_35yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'expanding', 35)
lstm_asymmetrical_expanding_35yr_32_16.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_asymmetrical_expanding_35yr_(32_16).csv')

In [ ]:
# lstm expanding 35yr (16,8) 
lstm_original_expanding_35yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_ori, 'original', variables, 'expanding', 35)
lstm_original_expanding_35yr_16_8.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_original_expanding_35yr_(16_8).csv')
lstm_squared_expanding_35yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_sqr, 'squared', variables_sq, 'expanding', 35)
lstm_squared_expanding_35yr_16_8.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_squared_expanding_35yr_(16_8).csv')
lstm_asymmetrical_expanding_35yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'expanding', 35)
lstm_asymmetrical_expanding_35yr_16_8.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_asymmetrical_expanding_35yr_(16_8).csv')

In [ ]:
# lstm moving 35yr (32,16,8) 
lstm_original_moving_35yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_ori, 'original', variables, 'moving', 35)
lstm_original_moving_35yr_32_16_8.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_original_moving_35yr_(32_16_8).csv')
lstm_squared_moving_35yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 35)
lstm_squared_moving_35yr_32_16_8.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_squared_moving_35yr_(32_16_8).csv')
lstm_asymmetrical_moving_35yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 35)
lstm_asymmetrical_moving_35yr_32_16_8.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_asymmetrical_moving_35yr_(32_16_8).csv')

In [ ]:
# lstm moving 35yr (32,16) 
lstm_original_moving_35yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_ori, 'original', variables, 'moving', 35)
lstm_original_moving_35yr_32_16.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_original_moving_35yr_(32_16).csv')
lstm_squared_moving_35yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 35)
lstm_squared_moving_35yr_32_16.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_squared_moving_35yr_(32_16).csv')
lstm_asymmetrical_moving_35yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 35)
lstm_asymmetrical_moving_35yr_32_16.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_asymmetrical_moving_35yr_(32_16).csv')

In [ ]:
# lstm moving 35yr (16,8) 
lstm_original_moving_35yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_ori, 'original', variables, 'moving', 35)
lstm_original_moving_35yr_16_8.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_original_moving_35yr_(16_8).csv')
lstm_squared_moving_35yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 35)
lstm_squared_moving_35yr_16_8.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_squared_moving_35yr_(16_8).csv')
lstm_asymmetrical_moving_35yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 35)
lstm_asymmetrical_moving_35yr_16_8.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_asymmetrical_moving_35yr_(16_8).csv')

In [ ]:
# lstm moving 20yr (32,16,8)
lstm_original_moving_20yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_ori, 'original', variables, 'moving', 20)
lstm_original_moving_20yr_32_16_8.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_original_moving_20yr_(32_16_8).csv')
lstm_squared_moving_20yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 20)
lstm_squared_moving_20yr_32_16_8.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_squared_moving_20yr_(32_16_8).csv')
lstm_asymmetrical_moving_20yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 20)
lstm_asymmetrical_moving_20yr_32_16_8.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_asymmetrical_moving_20yr_(32_16_8).csv')

In [ ]:
# lstm moving 20yr (32,16)
lstm_original_moving_20yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_ori, 'original', variables, 'moving', 20)
lstm_original_moving_20yr_32_16.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_original_moving_20yr_(32_16).csv')
lstm_squared_moving_20yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 20)
lstm_squared_moving_20yr_32_16.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_squared_moving_20yr_(32_16).csv')
lstm_asymmetrical_moving_20yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 20)
lstm_asymmetrical_moving_20yr_32_16.to_csv(r'C:\Users\Hightech\Desktop\thesis\result_ver_7\lstm\moving\lstm_asymmetrical_moving_20yr_(32_16).csv')

In [ ]:
# lstm moving 20yr (16,8)
lstm_original_moving_20yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_ori, 'original', variables, 'moving', 20)
lstm_original_moving_20yr_16_8.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\result_ver_7\lstm\moving\lstm_original_moving_20yr_(16_8).csv')
lstm_squared_moving_20yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 20)
lstm_squared_moving_20yr_16_8.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\result_ver_7\lstm\moving\lstm_squared_moving_20yr_(16_8).csv')
lstm_asymmetrical_moving_20yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 20)
lstm_asymmetrical_moving_20yr_16_8.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\result_ver_7\lstm\moving\lstm_asymmetrical_moving_20yr_(16_8).csv')

In [ ]:
# lstm moving 10yr (32,16,8) 
lstm_original_moving_10yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_ori, 'original', variables, 'moving', 10)
lstm_original_moving_10yr_32_16_8.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\result_ver_7\lstm\moving\lstm_original_moving_10yr_(32_16_8).csv')
lstm_squared_moving_10yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 10)
lstm_squared_moving_10yr_32_16_8.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\result_ver_7\lstm\moving\lstm_squared_moving_10yr_(32_16_8).csv')
lstm_asymmetrical_moving_10yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 10)
lstm_asymmetrical_moving_10yr_32_16_8.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\result_ver_7\lstm\moving\lstm_asymmetrical_moving_10yr_(32_16_8).csv')

In [ ]:
# lstm moving 10yr (32,16) 
lstm_original_moving_10yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_ori, 'original', variables, 'moving', 10)
lstm_original_moving_10yr_32_16.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\result_ver_7\lstm\moving\lstm_original_moving_10yr_(32_16).csv')
lstm_squared_moving_10yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 10)
lstm_squared_moving_10yr_32_16.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\result_ver_7\lstm\moving\lstm_squared_moving_10yr_(32_16).csv')
lstm_asymmetrical_moving_10yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 10)
lstm_asymmetrical_moving_10yr_32_16.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\result_ver_7\lstm\moving\lstm_asymmetrical_moving_10yr_(32_16).csv')

In [ ]:
# lstm moving 10yr (16,8) 
lstm_original_moving_10yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_ori, 'original', variables, 'moving', 10)
lstm_original_moving_10yr_16_8.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\result_ver_7\lstm\moving\lstm_original_moving_10yr_(16_8).csv')
lstm_squared_moving_10yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 10)
lstm_squared_moving_10yr_16_8.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\result_ver_7\lstm\moving\lstm_squared_moving_10yr_(16_8).csv')
lstm_asymmetrical_moving_10yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 10)
lstm_asymmetrical_moving_10yr_16_8.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\result_ver_7\lstm\moving\lstm_asymmetrical_moving_10yr_(16_8).csv')

In [ ]:
# Data Output only for Google CLoud Datalab
# lstm expanding 35yr (32,16,8) 
lstm_original_expanding_35yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_ori, 'original', variables, 'expanding', 35)
lstm_original_expanding_35yr_32_16_8.to_csv('lstm_original_expanding_35yr_(32_16_8).csv')
!gsutil cp 'lstm_original_expanding_35yr_(32_16_8).csv' 'gs://master_thesis_storage/lstm/expanding/lstm_original_expanding_35yr_(32_16_8).csv'

lstm_squared_expanding_35yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_sqr, 'squared', variables_sq, 'expanding', 35)
lstm_squared_expanding_35yr_32_16_8.to_csv('lstm_squared_expanding_35yr_(32_16_8).csv')
!gsutil cp 'lstm_squared_expanding_35yr_(32_16_8).csv' 'gs://master_thesis_storage/lstm/expanding/lstm_squared_expanding_35yr_(32_16_8).csv'

lstm_asymmetrical_expanding_35yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'expanding', 35)
lstm_asymmetrical_expanding_35yr_32_16_8.to_csv('lstm_asymmetrical_expanding_35yr_(32_16_8).csv')
!gsutil cp 'lstm_asymmetrical_expanding_35yr_(32_16_8).csv' 'gs://master_thesis_storage/lstm/expanding/lstm_asymmetrical_expanding_35yr_(32_16_8).csv'

# lstm expanding 35yr (32,16) 
lstm_original_expanding_35yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_ori, 'original', variables, 'expanding', 35)
lstm_original_expanding_35yr_32_16.to_csv('lstm_original_expanding_35yr_(32_16).csv')
!gsutil cp 'lstm_original_expanding_35yr_(32_16).csv' 'gs://master_thesis_storage/lstm/expanding/lstm_original_expanding_35yr_(32_16).csv'

lstm_squared_expanding_35yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_sqr, 'squared', variables_sq, 'expanding', 35)
lstm_squared_expanding_35yr_32_16.to_csv('lstm_squared_expanding_35yr_(32_16).csv')
!gsutil cp 'lstm_squared_expanding_35yr_(32_16).csv' 'gs://master_thesis_storage/lstm/expanding/lstm_squared_expanding_35yr_(32_16).csv'

lstm_asymmetrical_expanding_35yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'expanding', 35)
lstm_asymmetrical_expanding_35yr_32_16.to_csv('lstm_asymmetrical_expanding_35yr_(32_16).csv')
!gsutil cp 'lstm_asymmetrical_expanding_35yr_(32_16).csv' 'gs://master_thesis_storage/lstm/expanding/lstm_asymmetrical_expanding_35yr_(32_16).csv'

# lstm expanding 35yr (16,8) 
lstm_original_expanding_35yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_ori, 'original', variables, 'expanding', 35)
lstm_original_expanding_35yr_16_8.to_csv('lstm_original_expanding_35yr_(16_8).csv')
!gsutil cp 'lstm_original_expanding_35yr_(16_8).csv' 'gs://master_thesis_storage/lstm/expanding/lstm_original_expanding_35yr_(16_8).csv'

lstm_squared_expanding_35yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_sqr, 'squared', variables_sq, 'expanding', 35)
lstm_squared_expanding_35yr_16_8.to_csv('lstm_squared_expanding_35yr_(16_8).csv')
!gsutil cp 'lstm_squared_expanding_35yr_(16_8).csv' 'gs://master_thesis_storage/lstm/expanding/lstm_squared_expanding_35yr_(16_8).csv'

lstm_asymmetrical_expanding_35yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'expanding', 35)
lstm_asymmetrical_expanding_35yr_16_8.to_csv('lstm_asymmetrical_expanding_35yr_(16_8).csv')
!gsutil cp 'lstm_asymmetrical_expanding_35yr_(16_8).csv' 'gs://master_thesis_storage/lstm/expanding/lstm_asymmetrical_expanding_35yr_(16_8).csv'

# lstm moving 35yr (32,16,8) 
lstm_original_moving_35yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_ori, 'original', variables, 'moving', 35)
lstm_original_moving_35yr_32_16_8.to_csv('lstm_original_moving_35yr_(32_16_8).csv')
!gsutil cp 'lstm_original_moving_35yr_(32_16_8).csv' 'gs://master_thesis_storage/lstm/moving/lstm_original_moving_35yr_(32_16_8).csv'

lstm_squared_moving_35yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 35)
lstm_squared_moving_35yr_32_16_8.to_csv('lstm_squared_moving_35yr_(32_16_8).csv')
!gsutil cp 'lstm_squared_moving_35yr_(32_16_8).csv' 'gs://master_thesis_storage/lstm/moving/lstm_squared_moving_35yr_(32_16_8).csv'

lstm_asymmetrical_moving_35yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 35)
lstm_asymmetrical_moving_35yr_32_16_8.to_csv('lstm_asymmetrical_moving_35yr_(32_16_8).csv')
!gsutil cp 'lstm_asymmetrical_moving_35yr_(32_16_8).csv' 'gs://master_thesis_storage/lstm/moving/lstm_asymmetrical_moving_35yr_(32_16_8).csv'

# lstm moving 35yr (32,16) 
lstm_original_moving_35yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_ori, 'original', variables, 'moving', 35)
lstm_original_moving_35yr_32_16.to_csv('lstm_original_moving_35yr_(32_16).csv')
!gsutil cp 'lstm_original_moving_35yr_(32_16).csv' 'gs://master_thesis_storage/lstm/moving/lstm_original_moving_35yr_(32_16).csv'

lstm_squared_moving_35yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 35)
lstm_squared_moving_35yr_32_16.to_csv('lstm_squared_moving_35yr_(32_16).csv')
!gsutil cp 'lstm_squared_moving_35yr_(32_16).csv' 'gs://master_thesis_storage/lstm/moving/lstm_squared_moving_35yr_(32_16).csv'

lstm_asymmetrical_moving_35yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 35)
lstm_asymmetrical_moving_35yr_32_16.to_csv('lstm_asymmetrical_moving_35yr_(32_16).csv')
!gsutil cp 'lstm_asymmetrical_moving_35yr_(32_16).csv' 'gs://master_thesis_storage/lstm/moving/lstm_asymmetrical_moving_35yr_(32_16).csv'

# lstm moving 35yr (16,8) 
lstm_original_moving_35yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_ori, 'original', variables, 'moving', 35)
lstm_original_moving_35yr_16_8.to_csv('lstm_original_moving_35yr_(16_8).csv')
!gsutil cp 'lstm_original_moving_35yr_(16_8).csv' 'gs://master_thesis_storage/lstm/moving/lstm_original_moving_35yr_(16_8).csv'

lstm_squared_moving_35yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 35)
lstm_squared_moving_35yr_16_8.to_csv('lstm_squared_moving_35yr_(16_8).csv')
!gsutil cp 'lstm_squared_moving_35yr_(16_8).csv' 'gs://master_thesis_storage/lstm/moving/lstm_squared_moving_35yr_(16_8).csv'

lstm_asymmetrical_moving_35yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 35)
lstm_asymmetrical_moving_35yr_16_8.to_csv('lstm_asymmetrical_moving_35yr_(16_8).csv')
!gsutil cp 'lstm_asymmetrical_moving_35yr_(16_8).csv' 'gs://master_thesis_storage/lstm/moving/lstm_asymmetrical_moving_35yr_(16_8).csv'

# lstm moving 20yr (32,16,8)
lstm_original_moving_20yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_ori, 'original', variables, 'moving', 20)
lstm_original_moving_20yr_32_16_8.to_csv('lstm_original_moving_20yr_(32_16_8).csv')
!gsutil cp 'lstm_original_moving_20yr_(32_16_8).csv' 'gs://master_thesis_storage/lstm/moving/lstm_original_moving_20yr_(32_16_8).csv'

lstm_squared_moving_20yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 20)
lstm_squared_moving_20yr_32_16_8.to_csv('lstm_squared_moving_20yr_(32_16_8).csv')
!gsutil cp 'lstm_squared_moving_20yr_(32_16_8)csv' 'gs://master_thesis_storage/lstm/moving/lstm_squared_moving_20yr_(32_16_8).csv'

lstm_asymmetrical_moving_20yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 20)
lstm_asymmetrical_moving_20yr_32_16_8.to_csv('lstm_asymmetrical_moving_20yr_(32_16_8).csv')
!gsutil cp 'lstm_asymmetrical_moving_20yr_(32_16_8).csv' 'gs://master_thesis_storage/lstm/moving/lstm_asymmetrical_moving_20yr_(32_16_8).csv'

# lstm moving 20yr (32,16)
lstm_original_moving_20yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_ori, 'original', variables, 'moving', 20)
lstm_original_moving_20yr_32_16.to_csv('lstm_original_moving_20yr_(32_16).csv')
!gsutil cp 'lstm_original_moving_20yr_(32_16).csv' 'gs://master_thesis_storage/lstm/moving/lstm_original_moving_20yr_(32_16).csv'

lstm_squared_moving_20yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 20)
lstm_squared_moving_20yr_32_16.to_csv('lstm_squared_moving_20yr_(32_16).csv')
!gsutil cp 'lstm_squared_moving_20yr_(32_16)csv' 'gs://master_thesis_storage/lstm/moving/lstm_squared_moving_20yr_(32_16).csv'

lstm_asymmetrical_moving_20yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 20)
lstm_asymmetrical_moving_20yr_32_16.to_csv('lstm_asymmetrical_moving_20yr_(32_16).csv')
!gsutil cp 'lstm_asymmetrical_moving_20yr_(32_16).csv' 'gs://master_thesis_storage/lstm/moving/lstm_asymmetrical_moving_20yr_(32_16).csv'

# lstm moving 20yr (16,8)
lstm_original_moving_20yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_ori, 'original', variables, 'moving', 20)
lstm_original_moving_20yr_16_8.to_csv('lstm_original_moving_20yr_(16_8).csv')
!gsutil cp 'lstm_original_moving_20yr_(16_8).csv' 'gs://master_thesis_storage/lstm/moving/lstm_original_moving_20yr_(16_8).csv'

lstm_squared_moving_20yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 20)
lstm_squared_moving_20yr_16_8.to_csv('lstm_squared_moving_20yr_(16_8).csv')
!gsutil cp 'lstm_squared_moving_20yr_(16_8)csv' 'gs://master_thesis_storage/lstm/moving/lstm_squared_moving_20yr_(16_8).csv'

lstm_asymmetrical_moving_20yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 20)
lstm_asymmetrical_moving_20yr_16_8.to_csv('lstm_asymmetrical_moving_20yr_(16_8).csv')
!gsutil cp 'lstm_asymmetrical_moving_20yr_(16_8).csv' 'gs://master_thesis_storage/lstm/moving/lstm_asymmetrical_moving_20yr_(16_8).csv'

# lstm moving 10yr (32,16,8)
lstm_original_moving_10yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_ori, 'original', variables, 'moving', 10)
lstm_original_moving_10yr_32_16_8.to_csv('lstm_original_moving_10yr_(32_16_8).csv')
!gsutil cp 'lstm_original_moving_10yr_(32_16_8).csv' 'gs://master_thesis_storage/lstm/moving/lstm_original_moving_10yr_(32_16_8).csv'

lstm_squared_moving_10yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 10)
lstm_squared_moving_10yr_32_16_8.to_csv('lstm_squared_moving_10yr_(32_16_8).csv')
!gsutil cp 'lstm_squared_moving_10yr_(32_16_8)csv' 'gs://master_thesis_storage/lstm/moving/lstm_squared_moving_10yr_(32_16_8).csv'

lstm_asymmetrical_moving_10yr_32_16_8 = nn_prediction((32,16,8),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 10)
lstm_asymmetrical_moving_10yr_32_16_8.to_csv('lstm_asymmetrical_moving_10yr_(32_16_8).csv')
!gsutil cp 'lstm_asymmetrical_moving_10yr_(32_16_8).csv' 'gs://master_thesis_storage/lstm/moving/lstm_asymmetrical_moving_10yr_(32_16_8).csv'

# lstm moving 10yr (32,16)
lstm_original_moving_10yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_ori, 'original', variables, 'moving', 10)
lstm_original_moving_10yr_32_16.to_csv('lstm_original_moving_10yr_(32_16).csv')
!gsutil cp 'lstm_original_moving_10yr_(32_16).csv' 'gs://master_thesis_storage/lstm/moving/lstm_original_moving_10yr_(32_16).csv'

lstm_squared_moving_10yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 10)
lstm_squared_moving_10yr_32_16.to_csv('lstm_squared_moving_10yr_(32_16).csv')
!gsutil cp 'lstm_squared_moving_10yr_(32_16)csv' 'gs://master_thesis_storage/lstm/moving/lstm_squared_moving_10yr_(32_16).csv'

lstm_asymmetrical_moving_10yr_32_16 = nn_prediction((32,16),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 10)
lstm_asymmetrical_moving_10yr_32_16.to_csv('lstm_asymmetrical_moving_10yr_(32_16).csv')
!gsutil cp 'lstm_asymmetrical_moving_10yr_(32_16).csv' 'gs://master_thesis_storage/lstm/moving/lstm_asymmetrical_moving_10yr_(32_16).csv'

# lstm moving 10yr (16,8)
lstm_original_moving_10yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_ori, 'original', variables, 'moving', 10)
lstm_original_moving_10yr_16_8.to_csv('lstm_original_moving_10yr_(16_8).csv')
!gsutil cp 'lstm_original_moving_10yr_(16_8).csv' 'gs://master_thesis_storage/lstm/moving/lstm_original_moving_10yr_(16_8).csv'

lstm_squared_moving_10yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_sqr, 'squared', variables_sq, 'moving', 10)
lstm_squared_moving_10yr_16_8.to_csv('lstm_squared_moving_10yr_(16_8).csv')
!gsutil cp 'lstm_squared_moving_10yr_(16_8)csv' 'gs://master_thesis_storage/lstm/moving/lstm_squared_moving_10yr_(16_8).csv'

lstm_asymmetrical_moving_10yr_16_8 = nn_prediction((16,8),'LSTM',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 10)
lstm_asymmetrical_moving_10yr_16_8.to_csv('lstm_asymmetrical_moving_10yr_(16_8).csv')
!gsutil cp 'lstm_asymmetrical_moving_10yr_(16_8).csv' 'gs://master_thesis_storage/lstm/moving/lstm_asymmetrical_moving_10yr_(16_8).csv'


In [ ]:
# NN Expanding 35yr (32,16,8) 
nn_original_expanding_35yr_32_16_8 = nn_prediction((32,16,8),'MLP',mn_df_ori, 'original', variables, 'expanding', 35)
nn_original_expanding_35yr_32_16_8.to_csv('nn_original_expanding_35yr_(32_16_8).csv')
nn_squared_expanding_35yr_32_16_8 = nn_prediction((32,16,8),'MLP',mn_df_sqr, 'squared', variables_sq, 'expanding', 35)
nn_squared_expanding_35yr_32_16_8.to_csv('nn_squared_expanding_35yr_(32_16_8).csv')
nn_asymmetrical_expanding_35yr_32_16_8 = nn_prediction((32,16,8),'MLP',mn_df_asy, 'asymmetrical', variables_asy, 'expanding', 35)
nn_asymmetrical_expanding_35yr_32_16_8.to_csv('nn_asymmetrical_expanding_35yr_(32_16_8).csv')

In [ ]:
# NN Expanding 35yr (16,8,4)  
nn_original_expanding_35yr_16_8_4 = nn_prediction((16,8,4),'MLP',mn_df_ori, 'original', variables, 'expanding', 35)
nn_original_expanding_35yr_16_8_4.to_csv('nn_original_expanding_35yr_(16_8_4).csv')
nn_squared_expanding_35yr_16_8_4 = nn_prediction((16,8,4),'MLP',mn_df_sqr, 'squared', variables_sq, 'expanding', 35)
nn_squared_expanding_35yr_16_8_4.to_csv('nn_squared_expanding_35yr_(16_8_4).csv')
nn_asymmetrical_expanding_35yr_16_8_4 = nn_prediction((16,8,4),'MLP',mn_df_asy, 'asymmetrical', variables_asy, 'expanding', 35)
nn_asymmetrical_expanding_35yr_16_8_4.to_csv('nn_asymmetrical_expanding_35yr_(16_8_4).csv')

In [ ]:
# NN Expanding 35yr (16,8)  
nn_original_expanding_35yr_16_8 = nn_prediction((16,8),'MLP',mn_df_ori, 'original', variables, 'expanding', 35)
nn_original_expanding_35yr_16_8.to_csv('nn_original_expanding_35yr_(16_8).csv')
nn_squared_expanding_35yr_16_8 = nn_prediction((16,8),'MLP',mn_df_sqr, 'squared', variables_sq, 'expanding', 35)
nn_squared_expanding_35yr_16_8.to_csv('nn_squared_expanding_35yr_(16_8).csv')
nn_asymmetrical_expanding_35yr_16_8 = nn_prediction((16,8),'MLP',mn_df_asy, 'asymmetrical', variables_asy, 'expanding', 35)
nn_asymmetrical_expanding_35yr_16_8.to_csv('nn_asymmetrical_expanding_35yr_(16_8).csv')

In [ ]:
# NN Expanding 35yr (16)  
nn_original_expanding_35yr_16 = nn_prediction((16,),'MLP',mn_df_ori, 'original', variables, 'expanding', 35)
nn_original_expanding_35yr_16.to_csv('nn_original_expanding_35yr_(16).csv')
nn_squared_expanding_35yr_16 = nn_prediction((16,),'MLP',mn_df_sqr, 'squared', variables_sq, 'expanding', 35)
nn_squared_expanding_35yr_16.to_csv('nn_squared_expanding_35yr_(16).csv')
nn_asymmetrical_expanding_35yr_16 = nn_prediction((16,),'MLP',mn_df_asy, 'asymmetrical', variables_asy, 'expanding', 35)
nn_asymmetrical_expanding_35yr_16.to_csv('nn_asymmetrical_expanding_35yr_(16).csv')

In [ ]:
# NN moving 35yr (32,16,8)  
nn_original_moving_35yr_32_16_8 = nn_prediction((32,16,8),'MLP',mn_df_ori, 'original', variables, 'moving', 35)
nn_original_moving_35yr_32_16_8.to_csv(r'nn_original_moving_35yr_(32_16_8).csv')
nn_squared_moving_35yr_32_16_8 = nn_prediction((32,16,8),'MLP',mn_df_sqr, 'squared', variables_sq, 'moving', 35)
nn_squared_moving_35yr_32_16_8.to_csv(r'nn_squared_moving_35yr_(32_16_8).csv')
nn_asymmetrical_moving_35yr_32_16_8 = nn_prediction((32,16,8),'MLP',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 35)
nn_asymmetrical_moving_35yr_32_16_8.to_csv(r'nn_asymmetrical_moving_35yr_(32_16_8).csv')

In [ ]:
# NN moving 35yr (16,8,4)  
nn_original_moving_35yr_16_8_4 = nn_prediction((16,8,4),'MLP',mn_df_ori, 'original', variables, 'moving', 35)
nn_original_moving_35yr_16_8_4.to_csv(r'nn_original_moving_35yr_(16_8_4).csv')
nn_squared_moving_35yr_16_8_4 = nn_prediction((16,8,4),'MLP',mn_df_sqr, 'squared', variables_sq, 'moving', 35)
nn_squared_moving_35yr_16_8_4.to_csv(r'nn_squared_moving_35yr_(16_8_4).csv')
nn_asymmetrical_moving_35yr_16_8_4 = nn_prediction((16,8,4),'MLP',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 35)
nn_asymmetrical_moving_35yr_16_8_4.to_csv(r'nn_asymmetrical_moving_35yr_(16_8_4).csv')

In [ ]:
# NN moving 35yr (16,8)  
nn_original_moving_35yr_16_8 = nn_prediction((16,8),'MLP',mn_df_ori, 'original', variables, 'moving', 35)
nn_original_moving_35yr_16_8.to_csv(r'nn_original_moving_35yr_(16_8).csv')
nn_squared_moving_35yr_16_8 = nn_prediction((16,8),'MLP',mn_df_sqr, 'squared', variables_sq, 'moving', 35)
nn_squared_moving_35yr_16_8.to_csv(r'nn_squared_moving_35yr_(16_8).csv')
nn_asymmetrical_moving_35yr_16_8 = nn_prediction((16,8),'MLP',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 35)
nn_asymmetrical_moving_35yr_16_8.to_csv(r'nn_asymmetrical_moving_35yr_(16_8).csv')

In [ ]:
# NN moving 35yr (16)  
nn_original_moving_35yr_16 = nn_prediction((16,),'MLP',mn_df_ori, 'original', variables, 'moving', 35)
nn_original_moving_35yr_16.to_csv(r'nn_original_moving_35yr_(16).csv')
nn_squared_moving_35yr_16 = nn_prediction((16,),'MLP',mn_df_sqr, 'squared', variables_sq, 'moving', 35)
nn_squared_moving_35yr_16.to_csv(r'nn_squared_moving_35yr_(16).csv')
nn_asymmetrical_moving_35yr_16 = nn_prediction((16,),'MLP',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 35)
nn_asymmetrical_moving_35yr_16.to_csv(r'nn_asymmetrical_moving_35yr_(16).csv')

In [ ]:
# NN moving 20yr (32,16,8)  
nn_original_moving_20yr_32_16_8 = nn_prediction((32,16,8),'MLP',mn_df_ori, 'original', variables, 'moving', 20)
nn_original_moving_20yr_32_16_8.to_csv(r'nn_original_moving_20yr_(32_16_8).csv')
nn_squared_moving_20yr_32_16_8 = nn_prediction((32,16,8),'MLP',mn_df_sqr, 'squared', variables_sq, 'moving', 20)
nn_squared_moving_20yr_32_16_8.to_csv(r'nn_squared_moving_20yr_(32_16_8).csv')
nn_asymmetrical_moving_20yr_32_16_8 = nn_prediction((32,16,8),'MLP',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 20)
nn_asymmetrical_moving_20yr_32_16_8.to_csv(r'nn_asymmetrical_moving_20yr_(32_16_8).csv')

In [ ]:
# NN moving 20yr (16,8,4) 
nn_original_moving_20yr_16_8_4 = nn_prediction((16,8,4),'MLP',mn_df_ori, 'original', variables, 'moving', 20)
nn_original_moving_20yr_16_8_4.to_csv(r'nn_original_moving_20yr_(16_8_4).csv')
nn_squared_moving_20yr_16_8_4 = nn_prediction((16,8,4),'MLP',mn_df_sqr, 'squared', variables_sq, 'moving', 20)
nn_squared_moving_20yr_16_8_4.to_csv(r'nn_squared_moving_20yr_(16_8_4).csv')
nn_asymmetrical_moving_20yr_16_8_4 = nn_prediction((16,8,4),'MLP',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 20)
nn_asymmetrical_moving_20yr_16_8_4.to_csv(r'nn_asymmetrical_moving_20yr_(16_8_4).csv')

In [ ]:
# NN moving 20yr (16-8) 
nn_original_moving_20yr_16_8 = nn_prediction((16,8),'MLP',mn_df_ori, 'original', variables, 'moving', 20)
nn_original_moving_20yr_16_8.to_csv(r'nn_original_moving_20yr_(16_8).csv')
nn_squared_moving_20yr_16_8 = nn_prediction((16,8),'MLP',mn_df_sqr, 'squared', variables_sq, 'moving', 20)
nn_squared_moving_20yr_16_8.to_csv(r'nn_squared_moving_20yr_(16_8).csv')
nn_asymmetrical_moving_20yr_16_8 = nn_prediction((16,8),'MLP',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 20)
nn_asymmetrical_moving_20yr_16_8.to_csv(r'nn_asymmetrical_moving_20yr_(16_8).csv')

In [ ]:
# NN moving 20yr (16) 
nn_original_moving_20yr_16 = nn_prediction((16,),'MLP',mn_df_ori, 'original', variables, 'moving', 20)
nn_original_moving_20yr_16.to_csv(r'nn_original_moving_20yr_(16).csv')
nn_squared_moving_20yr_16 = nn_prediction((16,),'MLP',mn_df_sqr, 'squared', variables_sq, 'moving', 20)
nn_squared_moving_20yr_16.to_csv(r'nn_squared_moving_20yr_(16).csv')
nn_asymmetrical_moving_20yr_16 = nn_prediction((16,),'MLP',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 20)
nn_asymmetrical_moving_20yr_16.to_csv(r'nn_asymmetrical_moving_20yr_(16).csv')

In [ ]:
# NN moving 10yr (32,16,8) 
nn_original_moving_10yr_32_16_8 = nn_prediction((32,16,8),'MLP',mn_df_ori, 'original', variables, 'moving', 10)
nn_original_moving_10yr_32_16_8.to_csv(r'nn_original_moving_10yr_(32_16_8).csv')
nn_squared_moving_10yr_32_16_8 = nn_prediction((32,16,8),'MLP',mn_df_sqr, 'squared', variables_sq, 'moving', 10)
nn_squared_moving_10yr_32_16_8.to_csv(r'nn_squared_moving_10yr_(32_16_8).csv')
nn_asymmetrical_moving_10yr_32_16_8 = nn_prediction((32,16,8),'MLP',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 10)
nn_asymmetrical_moving_10yr_32_16_8.to_csv(r'nn_asymmetrical_moving_10yr_(32_16_8).csv')

In [ ]:
# NN moving 10yr (16,8,4) 
nn_original_moving_10yr_16_8_4 = nn_prediction((16,8,4),'MLP',mn_df_ori, 'original', variables, 'moving', 10)
nn_original_moving_10yr_16_8_4.to_csv(r'nn_original_moving_10yr_(16_8_4).csv')
nn_squared_moving_10yr_16_8_4 = nn_prediction((16,8,4),'MLP',mn_df_sqr, 'squared', variables_sq, 'moving', 10)
nn_squared_moving_10yr_16_8_4.to_csv(r'nn_squared_moving_10yr_(16_8_4).csv')
nn_asymmetrical_moving_10yr_16_8_4 = nn_prediction((16,8,4),'MLP',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 10)
nn_asymmetrical_moving_10yr_16_8_4.to_csv(r'nn_asymmetrical_moving_10yr_(16_8_4).csv')

In [ ]:
# NN moving 10yr (16,8) 
nn_original_moving_10yr_16_8 = nn_prediction((16,8),'MLP',mn_df_ori, 'original', variables, 'moving', 10)
nn_original_moving_10yr_16_8.to_csv(r'nn_original_moving_10yr_(16_8).csv')
nn_squared_moving_10yr_16_8 = nn_prediction((16,8),'MLP',mn_df_sqr, 'squared', variables_sq, 'moving', 10)
nn_squared_moving_10yr_16_8.to_csv(r'nn_squared_moving_10yr_(16_8).csv')
nn_asymmetrical_moving_10yr_16_8 = nn_prediction((16,8),'MLP',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 10)
nn_asymmetrical_moving_10yr_16_8.to_csv(r'nn_asymmetrical_moving_10yr_(16_8).csv')

In [ ]:
# NN moving 10yr (16) 
nn_original_moving_10yr_16 = nn_prediction((16,),'MLP',mn_df_ori, 'original', variables, 'moving', 10)
nn_original_moving_10yr_16.to_csv(r'nn_original_moving_10yr_(16).csv')
nn_squared_moving_10yr_16 = nn_prediction((16,),'MLP',mn_df_sqr, 'squared', variables_sq, 'moving', 10)
nn_squared_moving_10yr_16.to_csv(r'nn_squared_moving_10yr_(16).csv')
nn_asymmetrical_moving_10yr_16 = nn_prediction((16,),'MLP',mn_df_asy, 'asymmetrical', variables_asy, 'moving', 10)
nn_asymmetrical_moving_10yr_16.to_csv(r'nn_asymmetrical_moving_10yr_(16).csv')